In [2]:
from google.colab import drive 
drive.mount('/gdrive')




Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
targets_file = '/gdrive/My Drive/MSC_Thesis_Project/Data/target_labels.npy'
train_val_test_indices_file = '/gdrive/My Drive/MSC_Thesis_Project/Data/train_val_test_indices.json'  

features_file = {}
features_file['handcrafted'] = '/gdrive/My Drive/MSC_Thesis_Project/Data/handcrafted_features.csv'
features_file['word2vec'] = '/gdrive/My Drive/MSC_Thesis_Project/Data/word2vec_mean_features.npy'
features_file['doc2vec'] = '/gdrive/My Drive/MSC_Thesis_Project/Data/doc2vec_apnews_features.npy'
features_file['bert'] = '/gdrive/My Drive/MSC_Thesis_Project/Data/bert_features.npy'

ml_model = {}
ml_model['knn'] = {'name':'knn','k_range':[1,3,5,7,9,11,13,15,17,19,21]}
ml_model['svm'] = {'name':'svm','C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf','poly', 'sigmoid'], 'degree': [1,2,3,4,5]} # 
ml_model['xgboost'] = {'name':'xgboost'}
ml_model['lightgbm'] = {'name':'lightgbm'}
ml_model['mlp'] = {'name':'mlp','h':[[0.75,0.50,0.25,0.20],[0.75,0.25,0.15],[0.50,0.25,0.15]]}

In [4]:
!pip install import_ipynb
!pip install ipynb



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached import_ipynb-0.1.4-py3-none-any.whl (4.1 kB)
     |████████████████████████████████| 793 kB 5.6 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 47.9 MB/s 
     |████████████████████████████████| 106 kB 55.2 MB/s 
     |████████████████████████████████| 1.6 MB 20.8 MB/s 
     |████████████████████████████████| 381 kB 37.8 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 100 kB 8.0 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 87 kB 6.9 MB/s 
     |████████████████████████████████| 60 kB 7.8 MB/s 
     |████████████████████████████████| 117 kB 63.1 MB/s 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ipynb-0.5.1-py3-none-any.whl (6.9 kB)


In [ ]:
# %cd "/gdrive/My Drive/MSC_Thesis_Project/SRC/"
# %run get_train_val_test_features.ipynb
# import numpy as np

# results_list = []

# for f in features_file:       
#   train_feats,train_labels,val_feats,val_labels,test_feats,test_labels = get_features(features_file[f],targets_file, train_val_test_indices_file)
#   # combined train_val
#   trval_feats = np.vstack((train_feats,val_feats))
#   trval_labels = np.hstack((train_labels,val_labels))  
#   for m in ml_model:    
#     results_summ = {}
#     best_val_scores,best_val_model_hyperparams = train_and_eval(m,ml_model[m],train_feats,train_labels,val_feats,val_labels)
#     #print("features="+f+", model="+m+", best val = "+str(best_val_scores)+",  best params = "+ str(best_val_model_params))             
#     final_test_score,final_model = test_on_combined_train_val(m,best_val_model_hyperparams,trval_feats,trval_labels,test_feats,test_labels) 
#     print("FINAL:features="+f+", model="+m+", TEST F1= "+str(final_test_score))  
#     results_summ['model'] = m
#     results_summ['feature'] = f
#     results_summ['hyperparams'] = best_val_model_hyperparams
#     results_summ['model_params'] = final_model
#     results_summ['score'] = final_test_score
#     results_list.append(results_summ)
#   print("============")      
    


In [5]:
%cd "/gdrive/My Drive/MSC_Thesis_Project/SRC/"
%run get_train_val_test_features.ipynb
for f in features_file:       
    train_feats,train_labels,val_feats,val_labels,test_feats,test_labels = get_features(features_file[f],targets_file, train_val_test_indices_file)
    print('features='+f+' '+str(train_feats.shape))

/gdrive/My Drive/MSC_Thesis_Project/SRC
features=handcrafted (1026, 12)
features=word2vec (1026, 300)
features=doc2vec (1026, 300)
features=bert (1026, 768)


In [14]:
# ALTERNATE: Outer loop is ML models (classifiers)
%cd "/gdrive/My Drive/MSC_Thesis_Project/SRC/"
%run get_train_val_test_features.ipynb
import numpy as np

results_list = []

for m in ml_model: 
  for f in features_file:       
    train_feats,train_labels,val_feats,val_labels,test_feats,test_labels = get_features(features_file[f],targets_file, train_val_test_indices_file)
    
    # combined train_val
    trval_feats = np.vstack((train_feats,val_feats))
    trval_labels = np.hstack((train_labels,val_labels))  
     
    results_summ = {}
    best_val_scores,best_val_model_hyperparams = train_and_eval(m,ml_model[m],train_feats,train_labels,val_feats,val_labels)
    print("features="+f+", model="+m+", best val = "+fmtF1Score(best_val_scores)+",  best params = "+ str(best_val_model_hyperparams))             
    
    final_test_score,final_model = test_on_combined_train_val(m,best_val_model_hyperparams,trval_feats,trval_labels,test_feats,test_labels) 
    print("FINAL:features="+f+", model="+m+", TEST F1= "+fmtF1Score(final_test_score))  
    
    results_summ['model'] = m
    results_summ['feature'] = f
    results_summ['hyperparams'] = best_val_model_hyperparams
    results_summ['model_params'] = final_model
    results_summ['score'] = final_test_score
    results_list.append(results_summ)
    print("============")      
    


/gdrive/My Drive/MSC_Thesis_Project/SRC
KNN: k=1 f1 score=0.435
KNN: k=3 f1 score=0.443
KNN: k=5 f1 score=0.426
KNN: k=7 f1 score=0.456
KNN: k=9 f1 score=0.435
KNN: k=11 f1 score=0.464
KNN: k=13 f1 score=0.460
KNN: k=15 f1 score=0.460
KNN: k=17 f1 score=0.451
KNN: k=19 f1 score=0.447
KNN: k=21 f1 score=0.426
features=handcrafted, model=knn, best val = 0.464,  best params = 11
[[86 53  0]
 [66 68  2]
 [17 23  3]]
FINAL:features=handcrafted, model=knn, TEST F1= 0.494
KNN: k=1 f1 score=0.692
KNN: k=3 f1 score=0.667
KNN: k=5 f1 score=0.667
KNN: k=7 f1 score=0.629
KNN: k=9 f1 score=0.654
KNN: k=11 f1 score=0.675
KNN: k=13 f1 score=0.662
KNN: k=15 f1 score=0.641
KNN: k=17 f1 score=0.667
KNN: k=19 f1 score=0.646
KNN: k=21 f1 score=0.684
features=word2vec, model=knn, best val = 0.692,  best params = 1
[[87 47  5]
 [26 94 16]
 [ 2 16 25]]
FINAL:features=word2vec, model=knn, TEST F1= 0.648
KNN: k=1 f1 score=0.536
KNN: k=3 f1 score=0.515
KNN: k=5 f1 score=0.502
KNN: k=7 f1 score=0.502
KNN: k=9 f1

In [19]:
! pip install -U imbalanced-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)
/bin/bash: conda: command not found


In [6]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [7]:
!conda install -c conda-forge imbalanced-learn

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.



In [10]:
# ALTERNATE-2: Outer loop is ML models (classifiers)
# USING OVERSAMPLING/UNDERSAMPLING

%cd "/gdrive/My Drive/MSC_Thesis_Project/SRC/"
%run get_train_val_test_features.ipynb
import numpy as np

#from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE

smote=SMOTE("minority")

results_list = []

for m in ml_model: 
  for f in features_file:       
    train_feats,train_labels,val_feats,val_labels,test_feats,test_labels = get_features(features_file[f],targets_file, train_val_test_indices_file)
    
    X_train_rus, y_train_rus = smote.fit_resample(train_feats, train_labels)
    train_feats = X_train_rus
    train_labels = y_train_rus
    
    # combined train_val
    trval_feats = np.vstack((train_feats,val_feats))
    trval_labels = np.hstack((train_labels,val_labels))  

    X_train_rus, y_train_rus = smote.fit_resample(trval_feats, trval_labels)
    trval_feats = X_train_rus
    trval_labels = y_train_rus
     
    results_summ = {}
    best_val_scores,best_val_model_hyperparams = train_and_eval(m,ml_model[m],train_feats,train_labels,val_feats,val_labels)
    print("features="+f+", model="+m+", best val = "+fmtF1Score(best_val_scores)+",  best params = "+ str(best_val_model_hyperparams))             
    
    final_test_score,final_model = test_on_combined_train_val(m,best_val_model_hyperparams,trval_feats,trval_labels,test_feats,test_labels) 
    print("FINAL:features="+f+", model="+m+", TEST F1= "+fmtF1Score(final_test_score))  
    
    results_summ['model'] = m
    results_summ['feature'] = f
    results_summ['hyperparams'] = best_val_model_hyperparams
    results_summ['model_params'] = final_model
    results_summ['score'] = final_test_score
    results_list.append(results_summ)
    print("============")      
    


/gdrive/My Drive/MSC_Thesis_Project/SRC


/usr/local/lib/python3.7/site-packages/imblearn/utils/_validation.py:638: FutureWarning: Pass sampling_strategy=minority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


KNN: k=1 f1 score=0.418
KNN: k=3 f1 score=0.392
KNN: k=5 f1 score=0.380
KNN: k=7 f1 score=0.405
KNN: k=9 f1 score=0.329
KNN: k=11 f1 score=0.363
KNN: k=13 f1 score=0.359
KNN: k=15 f1 score=0.350
KNN: k=17 f1 score=0.342
KNN: k=19 f1 score=0.354
KNN: k=21 f1 score=0.380
features=handcrafted, model=knn, best val = 0.418,  best params = 1
[[64 54 21]
 [47 68 21]
 [15 17 11]]
FINAL:features=handcrafted, model=knn, TEST F1= 0.450
KNN: k=1 f1 score=0.637
KNN: k=3 f1 score=0.544
KNN: k=5 f1 score=0.544
KNN: k=7 f1 score=0.502
KNN: k=9 f1 score=0.498
KNN: k=11 f1 score=0.485
KNN: k=13 f1 score=0.464
KNN: k=15 f1 score=0.447
KNN: k=17 f1 score=0.447
KNN: k=19 f1 score=0.430
KNN: k=21 f1 score=0.405
features=word2vec, model=knn, best val = 0.637,  best params = 1
[[80 39 20]
 [22 67 47]
 [ 2  6 35]]
FINAL:features=word2vec, model=knn, TEST F1= 0.572
KNN: k=1 f1 score=0.489
KNN: k=3 f1 score=0.435
KNN: k=5 f1 score=0.405
KNN: k=7 f1 score=0.363
KNN: k=9 f1 score=0.295
KNN: k=11 f1 score=0.274
KNN

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

ensemble_results_list = []

for f in features_file:       
  train_feats,train_labels,val_feats,val_labels,test_feats,test_labels = get_features(features_file[f],targets_file, train_val_test_indices_file)
  # combined train_val
  trval_feats = np.vstack((train_feats,val_feats))
  trval_labels = np.hstack((train_labels,val_labels))  

  svm_model = []
  xgboost_model = []
  lightgbm_model = []
  mlp_model = []
  knn_model = []
  ensemble_results_summ = {}
  for r in results_list:
    results_summ = r
    if(results_summ['feature'] == f):    
      if(results_summ['model'] == 'knn'):        
        knn_model = results_summ['model_params']        
      if(results_summ['model'] == 'svm'):        
        svm_model = results_summ['model_params']
      if(results_summ['model'] == 'xgboost'):        
        xgboost_model = results_summ['model_params']
      if(results_summ['model'] == 'lightgbm'):        
        lightgbm_model = results_summ['model_params']
      if(results_summ['model'] == 'mlp'):
        mlp_model = results_summ['model_params']

  base_learners = [('knn',knn_model),('svm',svm_model),('xgboost',xgboost_model),('lightgbm',lightgbm_model),('mlp',mlp_model)]
      
  model = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
  model.fit(train_feats,train_labels)    
  y_pred = model.predict(test_feats)      
  final_test_score =  f1_score(test_labels, y_pred,average='micro')     
  print("ENSEMBLE:features="+f+", TEST F1= "+fmtF1Score(final_test_score))  
  ensemble_results_summ['features'] = f
  ensemble_results_summ['test_score'] = final_test_score
  ensemble_results_list.append(ensemble_results_summ)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


ENSEMBLE:features=handcrafted, TEST F1= 0.579
ENSEMBLE:features=word2vec, TEST F1= 0.752
ENSEMBLE:features=doc2vec, TEST F1= 0.708
ENSEMBLE:features=bert, TEST F1= 0.676


In [ ]:
import _pickle as pkl

results_lists = [results_list,ensemble_results_list]

pkl.dump(results_lists, open( "/gdrive/My Drive/MSC_Thesis_Project/Data/final_results.p", "wb" ) )

results_lists  = pkl.load(open("/gdrive/My Drive/MSC_Thesis_Project/Data/final_results.p", "rb"))

results_lists


[[{'feature': 'handcrafted',
   'hyperparams': 11,
   'model': 'knn',
   'model_params': KNeighborsClassifier(n_neighbors=11),
   'score': 0.4937106918238994},
  {'feature': 'word2vec',
   'hyperparams': 1,
   'model': 'knn',
   'model_params': KNeighborsClassifier(n_neighbors=1),
   'score': 0.6477987421383647},
  {'feature': 'doc2vec',
   'hyperparams': 1,
   'model': 'knn',
   'model_params': KNeighborsClassifier(n_neighbors=1),
   'score': 0.4748427672955975},
  {'feature': 'bert',
   'hyperparams': 21,
   'model': 'knn',
   'model_params': KNeighborsClassifier(n_neighbors=21),
   'score': 0.6289308176100629},
  {'feature': 'handcrafted',
   'hyperparams': {'C': 1, 'degree': 5, 'gamma': 0.001, 'kernel': 'poly'},
   'model': 'svm',
   'model_params': SVC(degree=5, kernel='poly'),
   'score': 0.48742138364779874},
  {'feature': 'word2vec',
   'hyperparams': {'C': 10, 'gamma': 1, 'kernel': 'rbf'},
   'model': 'svm',
   'model_params': SVC(C=10, gamma=1, probability=True),
   'score': 